In [11]:
import pandas as pd
import plotly.express as px
import folium

# Reading Proccessed data

In [12]:
city_info = pd.read_csv('./Datak/Processed_data/city_info.csv')
df = pd.read_pickle('./Datak/Processed_data/compiled_data.pkl')

In [13]:
df

,Unnamed: 0,ID,Date,tmax,tmin,prcp,Name,Lat,Lon
0,0,USC00042863,1894-01-01,60.0,41.0,0.00,Reno,39.4839,-119.7711
1,1,USC00042863,1894-01-02,58.0,50.0,0.40,Reno,39.4839,-119.7711
2,2,USC00042863,1894-01-03,57.0,42.0,0.00,Reno,39.4839,-119.7711
3,3,USC00042863,1894-01-04,53.0,42.0,0.28,Reno,39.4839,-119.7711
4,4,USC00042863,1894-01-05,50.0,38.0,0.00,Reno,39.4839,-119.7711
...,...,...,...,...,...,...,...,...,...
10475909,10475909,USW00094973,2021-12-27,32.0,1.2,0.22,Cheyenne,41.1519,-104.8061
10475910,10475910,USW00094973,2021-12-28,22.1,-5.8,0.06,Cheyenne,41.1519,-104.8061
10475911,10475911,USW00094973,2021-12-29,20.1,-4.9,0.00,Cheyenne,41.1519,-104.8061
10475912,10475912,USW00094973,2021-12-30,NaN,NaN,NaN,Cheyenne,41.1519,-104.8061


In [14]:
city_info.head()

,Unnamed: 0,Name,ID,Lat,Lon
0,0,Lander,USW00024021,42.8153,-108.7261
1,1,Lander,USW00024018,42.8153,-108.7261
2,2,Cheyenne,USW00014897,41.1519,-104.8061
3,3,Cheyenne,USW00094973,41.1519,-104.8061
4,4,Wausau,USW00014991,44.9258,-89.6256


In [25]:
city_info['Name']

0           Lander
1           Lander
2         Cheyenne
3         Cheyenne
4           Wausau
          ...     
205        Lincoln
206        Lincoln
207        Lincoln
208        Lincoln
209    GrandIsland
Name: Name, Length: 210, dtype: object

# Plotting the station points in different types of map for analyzing the functionality of each strategy

In [15]:
city_lat = city_info.Lat
city_Lon = city_info.Lon
d1 =city_info['Lat'][1]
d2 = city_info['Lon'][1]

In [30]:
m = folium.Map(location=[d1,d2], tiles='OpenStreetMap')
for i in range(len(city_info)):
    folium.Marker(
    [city_lat[i], city_Lon[i]],
    popup=f"{city_info.Name[i]}",
    fill=True,
    fill_color="#3186cc",
    size=5
).add_to(m)
    folium.Circle(
    [city_lat[i], city_Lon[i]],
    radius=50,
    popup=f"{city_info.Name[i]}",
    fill=True,
    fill_color="#3186cc",
    size=5
).add_to(m)


m